# Understanding GroupBy in Depth

In [16]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
%matplotlib inline
import datetime

In [17]:
df = pd.DataFrame({
    'product': ['a', 'b','a', 'c', 'a', 'b', 'b'],
    'sales': np.random.randint(10, 100, 7),
    'date': ['2020/01/15', '2020/01/15', '2020/01/12', '2020/01/30', '2020/02/05', '2020/02/06', '2020/02/15']
})
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df

,product,sales,date
0,a,71,2020-01-15
1,b,77,2020-01-15
2,a,70,2020-01-12
3,c,65,2020-01-30
4,a,46,2020-02-05
5,b,38,2020-02-06
6,b,30,2020-02-15


In [18]:
df.groupby(['product']).agg({'sales': sum})

,sales
product,
a,187
b,145
c,65


In [19]:
df.groupby(['product', 'date']).agg({'sales': sum})

sales
product date             
a       2020-01-12     70
        2020-01-15     71
        2020-02-05     46
b       2020-01-15     77
        2020-02-06     38
        2020-02-15     30
c       2020-01-30     65

## Custom aggregations

#### Calculate product wise average no. days between purchases

In [20]:
df

,product,sales,date
0,a,71,2020-01-15
1,b,77,2020-01-15
2,a,70,2020-01-12
3,c,65,2020-01-30
4,a,46,2020-02-05
5,b,38,2020-02-06
6,b,30,2020-02-15


In [21]:
for index, group in df.groupby(['product']):
    print(index)

a
b
c


In [22]:
for index, group in df.groupby(['product']):
    print(index)
    display(group)
    print('----')

a


,product,sales,date
0,a,71,2020-01-15
2,a,70,2020-01-12
4,a,46,2020-02-05


----
b


,product,sales,date
1,b,77,2020-01-15
5,b,38,2020-02-06
6,b,30,2020-02-15


----
c


,product,sales,date
3,c,65,2020-01-30


----


In [23]:
for index, group in df.groupby(['product']):
    print(index)
    group = group.sort_values('date')
    group['dates_diff'] = group['date'].diff()
    display(group)

a


,product,sales,date,dates_diff
2,a,70,2020-01-12,NaT
0,a,71,2020-01-15,3 days
4,a,46,2020-02-05,21 days


b


,product,sales,date,dates_diff
1,b,77,2020-01-15,NaT
5,b,38,2020-02-06,22 days
6,b,30,2020-02-15,9 days


c


,product,sales,date,dates_diff
3,c,65,2020-01-30,NaT


In [24]:
for index, group in df.groupby(['product']):
    print(index)
    group = group.sort_values('date')
    group['dates_diff'] = group['date'].diff().dt.days
    display(group)

a


,product,sales,date,dates_diff
2,a,70,2020-01-12,NaN
0,a,71,2020-01-15,3.0
4,a,46,2020-02-05,21.0


b


,product,sales,date,dates_diff
1,b,77,2020-01-15,NaN
5,b,38,2020-02-06,22.0
6,b,30,2020-02-15,9.0


c


,product,sales,date,dates_diff
3,c,65,2020-01-30,NaN


In [25]:
for index, group in df.groupby(['product']):
    group = group.sort_values('date')
    group['dates_diff'] = group['date'].diff().dt.days
    print(index, group['dates_diff'].mean())

a 12.0
b 15.5
c nan


In [26]:
def get_days_diff(group):
    group = group.sort_values('date')
    group['dates_diff'] = group['date'].diff().dt.days
    return group['dates_diff'].mean()

In [27]:
df.groupby(['product']).apply(get_days_diff)

product
a    12.0
b    15.5
c     NaN
dtype: float64

In [28]:
summary = df.groupby(['product']).apply(get_days_diff)
summary

product
a    12.0
b    15.5
c     NaN
dtype: float64

In [29]:
summary.reset_index()

,product,0
0,a,12.0
1,b,15.5
2,c,NaN


In [30]:
summary.name = 'avg_days_bwt_purchase'
summary.reset_index()

,product,avg_days_bwt_purchase
0,a,12.0
1,b,15.5
2,c,NaN
